# Promptmteo Usage - Save and Load Model

## 1. Create a model

### 1.1 Instance a DocumentClassifier

In [11]:
from promptmeteo import DocumentClassifier

clf = DocumentClassifier(
        language            = 'en',
        model_provider_name = 'hf_pipeline',
        model_name          = 'google/flan-t5-small',
        prompt_labels       = ['positive', 'negative']
    )

### 1.2 Prompt Templates

#### 1.2.1 Predefined prompt templates

Promptmeteo has predefined prompt templates for each of the availables languages, models and tasks. This allows the user not to wonder about defining the prefect prompt but rather, to parametrize the prompt with the parameters of the use case. For the document classification task in english with OpenAI, the prompt template is:

In [12]:
print(clf.task.prompt.PROMPT_EXAMPLE)

TEMPLATE:
    "I need you to help me with a text classification task.
    {__PROMPT_DOMAIN__}
    {__PROMPT_LABELS__}

    {__CHAIN_THOUGHT__}
    {__ANSWER_FORMAT__}"


PROMPT_DOMAIN:
    "The texts you will be processing are from the {__DOMAIN__} domain."


PROMPT_LABELS:
    "I want you to classify the texts into one of the following categories:
    {__LABELS__}."


PROMPT_DETAIL:
    ""


CHAIN_THOUGHT:
    "Please provide a step-by-step argument for your answer, explain why you
    believe your final choice is justified, and make sure to conclude your
    explanation with the name of the class you have selected as the correct
    one, in lowercase and without punctuation."


ANSWER_FORMAT:
    "In your response, include only the name of the class as a single word, in
    lowercase, without punctuation, and without adding any other statements or
    words."



&nbsp;

#### 1.2.2 Prompt text - Prompt injection

This template is used to build a prompt text by adding the stament bellow into the template. This technique is called **prompt injection**

In [14]:
print(clf.task.prompt.template)

I need you to help me with a text classification task.  I want you to classify the texts into one of the following categories: positive, negative.
Please provide a step-by-step argument for your answer, explain why you believe your final choice is justified, and make sure to conclude your explanation with the name of the class you have selected as the correct one, in lowercase and without punctuation. In your response, include only the name of the class as a single word, in lowercase, without punctuation, and without adding any other statements or words.


&nbsp;

#### 1.2.3 Injecting labels - Zero-shot Prompting

We can see tha not only the prompt has beenn build correctly, but also the labels `['positive', 'negative']` from the class inicialitations has been injected. Given the labels without more examples is called **Zero-shot prompting**

I need you to help me with a text classification task.  I want you to classify the texts into one of the following categories: **positive, negative.**
Please provide a step-by-step argument for your answer, explain why you believe your final choice is justified, and make sure to conclude your explanation with the name of the class you have selected as the correct one, in lowercase and without punctuation. In your response, include only the name of the class as a single word, in lowercase, without punctuation, and without adding any other statements or words.


&nbsp;


---

&nbsp;


## 2. Train a model

### 2.1 Train function

The model can be trained by giving examples to the function `train()`

In [16]:
clf = clf.train(
    examples    = ['i am happy', 'doesnt matter', 'I hate it'],
    annotations = ['positive', 'neutral', 'negative'],
)

&nbsp;


### 2.2 Examples Injection - Few-shot prompting

When a new sample cames, the examples are added to the prompt to help the model to improve the answers. This technique is called **Few-show prompting**. This examples are choosed rom those that are more related to the new sample passes for makein inference. We can see that now, the prompt with the examples has the following aspect

In [22]:
print(clf.task._get_prompt('NEW EXAMPLE'))

 I need you to help me with a text classification task.  I want you to classify the texts into one of the following categories: positive, negative.
Please provide a step-by-step argument for your answer, explain why you believe your final choice is justified, and make sure to conclude your explanation with the name of the class you have selected as the correct one, in lowercase and without punctuation. In your response, include only the name of the class as a single word, in lowercase, without punctuation, and without adding any other statements or words.

Ejemplo: I hate it
Respuesta: negative

Ejemplo: doesnt matter
Respuesta: neutral

Ejemplo: i am happy
Respuesta: positive

Ejemplo: NEW EXAMPLE
Respuesta: 


&nbsp;


### 2.3 Save Model - FAISS

The examples given to the function train are saved in a vectorstore in local (with FAISS) and this vectorstore can be serialized to disk. Saving this examples in disk allow us to easily reuse it from new use cases without having to recover the original data again

In [25]:
clf.save_model('my_classifier.meteo')

&nbsp;


---

&nbsp;


## 3. Load a model

### 3.1 Model creation

For reloading a model we have to intanciate a `DocumentClassifier` and use the function `load_model()`

In [27]:
from promptmeteo import DocumentClassifier

new_clf = DocumentClassifier(
        language            = 'en',
        model_provider_name = 'hf_pipeline',
        model_name          = 'google/flan-t5-small',
        prompt_labels       = ['positive', 'negative']
    ).load_model('my_classifier.meteo')

&nbsp;


### 3.2 Predict new data

By calling the function `predict()` we can use the prompt created with the examples loaded again to predict the classification over new data

In [29]:
new_clf.predict(['so cool!!'])

[['positive']]

&nbsp;


---

&nbsp;


## 4 Conclusions

* In this example we have shown how Promptmeteo can be used as a machine learning frameworks such as Scikit-Learn o Pytorch. It has a similar interface with allows to save the results from the training in a binary file and reuse it. This eases the integration of LLM solution in ML pipeline tools such as Sagemaker or Vertex.

* Promptmeteo does not only include code data to simplify the integration to LLM model and services. It also include predefined Prompt Engineer logic for different models and tasks. It allows to focus on developing a solution rather than writting prompts, and ensures that the prompt has been tested correctly by Promptmeteo, which make this kind os solution less error-prone.